# Raspberry Pi

Raspberry Pi getting started: https://www.raspberrypi.com/documentation/computers/getting-started.html#install-an-operating-system
Create one SD card via Raspberry Pi Imager.

The raspi-config Tool: https://www.raspberrypi.com/documentation/computers/configuration.html#raspi-config

The raspi-config tool can also be run in a non-interactive mode: sudo raspi-config nonint <command> <arguments>

## GPIOs in Raspberry Pi

https://www.raspberrypi.com/documentation/computers/raspberry-pi.html#gpio-and-the-40-pin-header

A handy reference can be accessed on the Raspberry Pi by opening a terminal window and running the command pinout. This tool is provided by the GPIO Zero Python library, which is installed by default in Raspberry Pi OS.

Interactive pinout diagram: https://pinout.xyz

While connecting up simple components to the GPIO pins is perfectly safe, it’s important to be careful how you wire things up. LEDs should have resistors to limit the current passing through them. Do not use 5V for 3.3V components. Do not connect motors directly to the GPIO pins, instead use an H-bridge circuit or a motor controller board.

In order to use the GPIO ports your user must be a member of the gpio group. The pi user is a member by default, other users need to be added manually.

sudo usermod -a -G gpio <username>

In [1]:
!sudo usermod -a -G gpio ethan

Using the GPIO Zero library makes it easy to get started with controlling GPIO devices with Python. The library is comprehensively documented at gpiozero.readthedocs.io.

In [5]:
!sudo apt install python3-gpiozero

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-gpiozero is already the newest version (2.0-1).
0 upgraded, 0 newly installed, 0 to remove and 81 not upgraded.


In [3]:
!pip3 install gpiozero

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.5/150.5 kB 391.0 kB/s eta 0:00:00a 0:00:01


In [2]:
!pip install lgpio

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple


In [1]:
import RPi.GPIO as GPIO

In [4]:
!sudo apt-get install rpi.gpio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'python3-rpi.gpio' for regex 'rpi.gpio'
Note, selecting 'python-rpi.gpio' for regex 'rpi.gpio'
Note, selecting 'rpi.gpio-common' for regex 'rpi.gpio'
python3-rpi.gpio is already the newest version (0.7.1~a4-1+b4).
rpi.gpio-common is already the newest version (0.7.1~a4-1+b4).
rpi.gpio-common set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 81 not upgraded.


In [7]:
!sudo apt-get install python3-dev python3-rpi.gpio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-dev is already the newest version (3.11.2-1+b1).
python3-dev set to manually installed.
python3-rpi.gpio is already the newest version (0.7.1~a4-1+b4).
0 upgraded, 0 newly installed, 0 to remove and 81 not upgraded.


In [2]:
!pip3 install RPi.GPIO

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
  Preparing metadata (setup.py) ... done
  DEPRECATION: RPi.GPIO is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for RPi.GPIO ... done


In [3]:
import RPi.GPIO as GPIO

There are two ways of numbering the IO pins on a Raspberry Pi within RPi.GPIO. The first is using the BOARD numbering system. This refers to the pin numbers on the P1 header of the Raspberry Pi board. The advantage of using this numbering system is that your hardware will always work, regardless of the board revision of the RPi. You will not need to rewire your connector or change your code.

The second numbering system is the BCM numbers. This is a lower level way of working - it refers to the channel numbers on the Broadcom SOC. You have to always work with a diagram of which channel number goes to which pin on the RPi board. Your script could break between revisions of Raspberry Pi boards.

To specify which you are using using (mandatory):

GPIO.setmode(GPIO.BOARD)
  # or
GPIO.setmode(GPIO.BCM)

In [11]:
GPIO.setmode(GPIO.BCM) #activate the Broadcom-chip specific pin numbers.

In [12]:
mode = GPIO.getmode()
mode

11

To set a pin mode, use the setup([pin], [GPIO.IN, GPIO.OUT] function. 

In [4]:
GPIO.setup(18, GPIO.OUT) #pin18 as output

In [5]:
GPIO.output(18, GPIO.HIGH) #output high

Writing a pin to GPIO.HIGH will drive it to 3.3V, and GPIO.LOW will set it to 0V. For the lazy, alternative to GPIO.HIGH and GPIO.LOW, you can use either 1, True, 0 or False to set a pin value.

PWM ("Analog") Output:
To initialize PWM, use GPIO.PWM([pin], [frequency]) function. 
Then use pwm.start([duty cycle]) function to set an initial value.

In [6]:
#will set our PWM pin up with a frequency of 1kHz, and set that output to a 50% duty cycle.
pwm = GPIO.PWM(18, 1000)
pwm.start(50)

To adjust the value of the PWM output, use the pwm.ChangeDutyCycle([duty cycle]) function. [duty cycle] can be any value between 0 (i.e 0%/LOW) and 100 (ie.e 100%/HIGH). So to set a pin to 75% on, for example, you could write:

In [7]:
pwm.ChangeDutyCycle(75)

To turn PWM on that pin off, use the pwm.stop() command.

In [8]:
pwm.stop()

If a pin is configured as an input, you can use the GPIO.input([pin]) function to read its value. The input() function will return either a True or False indicating whether the pin is HIGH or LOW. You can use an if statement to test this

You need to set up every channel you are using as an input or an output. To configure a channel as an input:

GPIO.setup(channel, GPIO.IN)
(where channel is the channel number based on the numbering system you have specified (BOARD or BCM)).

In [14]:
GPIO.setup(17, GPIO.IN)

/tmp/ipykernel_11666/2004701586.py:1: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(17, GPIO.IN)


In [15]:
if GPIO.input(17):
    print("Pin 11 is HIGH")
else:
    print("Pin 11 is LOW")


Pin 11 is LOW


In [16]:
GPIO.cleanup() #To clean up at the end of your script

It is possible that don't want to clean up every channel leaving some set up when your program exits. You can clean up individual channels, a list or a tuple of channels:

GPIO.cleanup(channel)
GPIO.cleanup( (channel1, channel2) )
GPIO.cleanup( [channel1, channel2] )

In [17]:
GPIO.RPI_INFO

{'P1_REVISION': 3,
 'REVISION': 'c03112',
 'TYPE': 'Pi 4 Model B',
 'MANUFACTURER': 'Sony UK',
 'PROCESSOR': 'BCM2711',
 'RAM': '4G'}

In [18]:
GPIO.RPI_INFO['P1_REVISION']

3

In [19]:
GPIO.VERSION

'0.7.1'

### gpiozero

In [2]:
from gpiozero import LED

In [4]:
from gpiozero import LED
from time import sleep

led = LED(17) #control an LED connected to GPIO17

while True:
    led.on()
    sleep(1)
    led.off()
    sleep(1)

/home/ethan/mypyvenv/lib/python3.11/site-packages/gpiozero/devices.py:295: PinFactoryFallback: Falling back from lgpio: module 'lgpio' has no attribute 'SET_BIAS_DISABLE'
  warnings.warn(


KeyboardInterrupt: 

In [5]:
from gpiozero import Button
from time import sleep

button = Button(2)

while True:
    if button.is_pressed:
        print("Pressed")
    else:
        print("Released")
    sleep(1)

Released
Released
Released
Released
Released
Released
Released
Released
Released
Released
Released
Released


KeyboardInterrupt: 

## I2C in Raspberry Pi

Install I2C tools: #already installed
sudo apt install python3-smbus 
sudo apt-get install -y i2c-tools

In [6]:
!lsmod | grep i2c_

i2c_dev                20480  0
i2c_brcmstb            16384  0
i2c_bcm2835            20480  0


In [20]:
!ls /dev/*i2c*

/dev/i2c-1  /dev/i2c-20  /dev/i2c-21


In [ ]:
from os import listdir

In [22]:
!i2cdetect -y 1

     0  1  2  3  4  5  6  7  8  9  a  b  c  d  e  f
00:                         08 09 0a 0b 0c 0d 0e 0f 
10: 10 11 12 13 14 15 16 17 18 19 1a 1b 1c 1d 1e 1f 
20: 20 21 22 23 24 25 26 27 28 29 2a 2b 2c 2d 2e 2f 
30: 30 31 32 33 34 35 36 37 38 39 3a 3b 3c 3d 3e 3f 
40: 40 41 42 43 44 45 46 47 48 49 4a 4b 4c 4d 4e 4f 
50: 50 51 52 53 54 55 56 57 58 59 5a 5b 5c 5d 5e 5f 
60: 60 61 62 63 64 65 66 67 68 69 6a 6b 6c 6d 6e 6f 
70: 70 71 72 73 74 75 76 77                         


In [2]:
!pip3 install smbus2

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple


In [23]:
from smbus2 import SMBus, i2c_msg

In [28]:
#import smbus
import time
import math

In [25]:
bus_number=1
bus = SMBus(bus_number)

In [29]:
#ref: https://github.com/sunfounder/SunFounder_PiCar/blob/master/picar/SunFounder_PCA9685/PCA9685.py
class PWM(object):
    """A PWM control class for PCA9685."""
    _MODE1              = 0x00
    _MODE2              = 0x01
    _SUBADR1            = 0x02
    _SUBADR2            = 0x03
    _SUBADR3            = 0x04
    _PRESCALE           = 0xFE
    _LED0_ON_L          = 0x06
    _LED0_ON_H          = 0x07
    _LED0_OFF_L         = 0x08
    _LED0_OFF_H         = 0x09
    _ALL_LED_ON_L       = 0xFA
    _ALL_LED_ON_H       = 0xFB
    _ALL_LED_OFF_L      = 0xFC
    _ALL_LED_OFF_H      = 0xFD

    _RESTART            = 0x80
    _SLEEP              = 0x10
    _ALLCALL            = 0x01
    _INVRT              = 0x10
    _OUTDRV             = 0x04

    _DEBUG = False
    _DEBUG_INFO = 'DEBUG "PCA9685.py":'

    def __init__(self, bus_number=1, address=0x40):
        self.address = address
        self.bus_number = bus_number
        self.bus = SMBus(self.bus_number)

    def _debug_(self,message):
        if self._DEBUG:
            print(self._DEBUG_INFO,message)
    
    def setup(self):
        '''Init the class with bus_number and address'''
        self._debug_('Reseting PCA9685 MODE1 (without SLEEP) and MODE2')
        self.write_all_value(0, 0)
        self._write_byte_data(self._MODE2, self._OUTDRV)
        self._write_byte_data(self._MODE1, self._ALLCALL)
        time.sleep(0.005)

        mode1 = self._read_byte_data(self._MODE1)
        mode1 = mode1 & ~self._SLEEP
        self._write_byte_data(self._MODE1, mode1)
        time.sleep(0.005)
        self._frequency = 60

    def _write_byte_data(self, reg, value):
        '''Write data to I2C with self.address'''
        self._debug_('Writing value %2X to %2X' % (value, reg))
        try:
            self.bus.write_byte_data(self.address, reg, value)
        except Exception as i:
            print(i)
            self._check_i2c()

    def _read_byte_data(self, reg):
        '''Read data from I2C with self.address'''
        self._debug_('Reading value from %2X' % reg)
        try:
            results = self.bus.read_byte_data(self.address, reg)
            return results
        except Exception as i:
            print(i)
            self._check_i2c()

    def _run_command(self, cmd):
        import subprocess
        p = subprocess.Popen(
            cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        result = p.stdout.read().decode('utf-8')
        status = p.poll()
        # print(result)
        # print(status)
        return status, result

    def _check_i2c(self):
        from os import listdir
        print("I2C bus number is: %s" % self.bus_number)
        print("Checking I2C device:")
        devices = listdir("/dev/")
        if "i2c-%d" % self.bus_number in devices:
            print("I2C device exist.")
        else:
            print("Seems like I2C have not been set, run 'sudo raspi-config' to enable I2C")
        cmd = "i2cdetect -y %s" % self.bus_number
        _, output = self._run_command(cmd)
        print("Your PCA9685 address is set to 0x%02X" % self.address)
        print("i2cdetect output:")
        print(output)
        outputs = output.split('\n')[1:]
        addresses = []
        for tmp_addresses in outputs:
            tmp_addresses = tmp_addresses.split(':')
            if len(tmp_addresses) < 2:
                continue
            else:
                tmp_addresses = tmp_addresses[1]
            tmp_addresses = tmp_addresses.strip().split(' ')
            for address in tmp_addresses:
                if address != '--':
                    addresses.append(address)
        print("Conneceted i2c device:")
        if addresses == []:
            print("None")
        else:
            for address in addresses:
                print("  0x%s" % address)
        if "%02X" % self.address in addresses:
            print("Wierd, I2C device is connected, Try to run the program again, If problem stills, email this information to support@sunfounder.com")
        else:
            print("Device is missing.")
            print("Check the address or wiring of PCA9685 Server driver, or email this information to support@sunfounder.com")
            quit()

    @property
    def frequency(self):
        return self._frequency

    @frequency.setter
    def frequency(self, freq):
        '''Set PWM frequency'''
        self._debug_('Set frequency to %d' % freq)
        self._frequency = freq
        prescale_value = 25000000.0
        prescale_value /= 4096.0
        prescale_value /= float(freq)
        prescale_value -= 1.0
        self._debug_('Setting PWM frequency to %d Hz' % freq)
        self._debug_('Estimated pre-scale: %d' % prescale_value)
        prescale = math.floor(prescale_value + 0.5)
        self._debug_('Final pre-scale: %d' % prescale)

        old_mode = self._read_byte_data(self._MODE1);
        new_mode = (old_mode & 0x7F) | 0x10
        self._write_byte_data(self._MODE1, new_mode)
        self._write_byte_data(self._PRESCALE, int(math.floor(prescale)))
        self._write_byte_data(self._MODE1, old_mode)
        time.sleep(0.005)
        self._write_byte_data(self._MODE1, old_mode | 0x80)

    def write(self, channel, on, off):
        '''Set on and off value on specific channel'''
        self._debug_('Set channel "%d" to value "%d"' % (channel, off))
        self._write_byte_data(self._LED0_ON_L+4*channel, on & 0xFF)
        self._write_byte_data(self._LED0_ON_H+4*channel, on >> 8)
        self._write_byte_data(self._LED0_OFF_L+4*channel, off & 0xFF)
        self._write_byte_data(self._LED0_OFF_H+4*channel, off >> 8)

    def write_all_value(self, on, off):
        '''Set on and off value on all channel'''
        self._debug_('Set all channel to value "%d"' % (off))
        self._write_byte_data(self._ALL_LED_ON_L, on & 0xFF)
        self._write_byte_data(self._ALL_LED_ON_H, on >> 8)
        self._write_byte_data(self._ALL_LED_OFF_L, off & 0xFF)
        self._write_byte_data(self._ALL_LED_OFF_H, off >> 8)

    def map(self, x, in_min, in_max, out_min, out_max):
        '''To map the value from arange to another'''
        return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

    @property
    def debug(self):
        return self._DEBUG

    @debug.setter
    def debug(self, debug):
        '''Set if debug information shows'''
        if debug in (True, False):
            self._DEBUG = debug
        else:
            raise ValueError('debug must be "True" (Set debug on) or "False" (Set debug off), not "{0}"'.format(debug))

        if self._DEBUG:
            print(self._DEBUG_INFO, "Set debug on")
        else:
            print(self._DEBUG_INFO, "Set debug off")

In [30]:
import time
pwm = PWM()
pwm.frequency = 60

In [32]:
for i in range(16):
    time.sleep(0.5)
    print('\nChannel %d\n' % i)
    time.sleep(0.5)
    for j in range(16):
        pwm.write(i, 0, j)
        print('PWM value: %d' % j)
        time.sleep(0.0003)


Channel 0

PWM value: 0
PWM value: 1
PWM value: 2
PWM value: 3
PWM value: 4
PWM value: 5
PWM value: 6
PWM value: 7
PWM value: 8
PWM value: 9
PWM value: 10
PWM value: 11
PWM value: 12
PWM value: 13
PWM value: 14
PWM value: 15

Channel 1

PWM value: 0
PWM value: 1
PWM value: 2
PWM value: 3
PWM value: 4
PWM value: 5
PWM value: 6
PWM value: 7
PWM value: 8
PWM value: 9
PWM value: 10
PWM value: 11
PWM value: 12
PWM value: 13
PWM value: 14
PWM value: 15

Channel 2

PWM value: 0
PWM value: 1
PWM value: 2
PWM value: 3
PWM value: 4
PWM value: 5
PWM value: 6
PWM value: 7
PWM value: 8
PWM value: 9
PWM value: 10
PWM value: 11
PWM value: 12
PWM value: 13
PWM value: 14
PWM value: 15

Channel 3

PWM value: 0
PWM value: 1
PWM value: 2
PWM value: 3
PWM value: 4
PWM value: 5
PWM value: 6
PWM value: 7
PWM value: 8
PWM value: 9
PWM value: 10
PWM value: 11
PWM value: 12
PWM value: 13
PWM value: 14
PWM value: 15

Channel 4

PWM value: 0
PWM value: 1
PWM value: 2
PWM value: 3
PWM value: 4
PWM value: 5
PWM va

In [33]:
class Servo(object):
	'''Servo driver class'''
	_MIN_PULSE_WIDTH = 600
	_MAX_PULSE_WIDTH = 2400
	_DEFAULT_PULSE_WIDTH = 1500
	_FREQUENCY = 60

	_DEBUG = False
	_DEBUG_INFO = 'DEBUG "Servo.py":'

	def __init__(self, channel, offset=0, lock=True, bus_number=1, address=0x40):
		''' Init a servo on specific channel, this offset '''
		if channel<0 or channel > 16:
			raise ValueError("Servo channel \"{0}\" is not in (0, 15).".format(channel))
		self._debug_("Debug on")
		self.channel = channel
		self.offset = offset
		self.lock = lock

		self.pwm = PWM(bus_number=bus_number, address=address)
		self.frequency = self._FREQUENCY
		self.write(90)
	
	def _debug_(self,message):
		if self._DEBUG:
			print(self._DEBUG_INFO,message)

	def setup(self):
		self.pwm.setup()

	def _angle_to_analog(self, angle):
		''' Calculate 12-bit analog value from giving angle '''
		pulse_wide   = self.pwm.map(angle, 0, 180, self._MIN_PULSE_WIDTH, self._MAX_PULSE_WIDTH)
		analog_value = int(float(pulse_wide) / 1000000 * self.frequency * 4096)
		self._debug_('Angle %d equals Analog_value %d' % (angle, analog_value))
		return analog_value

	@property
	def frequency(self):
		return self._frequency

	@frequency.setter
	def frequency(self, value):
		self._frequency = value
		self.pwm.frequency = value

	@property
	def offset(self):
		return self._offset

	@offset.setter
	def offset(self, value):
		''' Set offset for much user-friendly '''
		self._offset = value
		self._debug_('Set offset to %d' % self.offset)

	def write(self, angle):
		''' Turn the servo with giving angle. '''
		if self.lock:
			if angle > 180:
				angle = 180
			if angle < 0:
				angle = 0
		else:
			if angle<0 or angle>180:
				raise ValueError("Servo \"{0}\" turn angle \"{1}\" is not in (0, 180).".format(self.channel, angle))
		val = self._angle_to_analog(angle)
		val += self.offset
		self.pwm.write(self.channel, 0, val)
		self._debug_('Turn angle = %d' % angle)

	@property
	def debug(self):
		return self._DEBUG

	@debug.setter
	def debug(self, debug):
		''' Set if debug information shows '''
		if debug in (True, False):
			self._DEBUG = debug
		else:
			raise ValueError('debug must be "True" (Set debug on) or "False" (Set debug off), not "{0}"'.format(debug))

		if self._DEBUG:
			print(self._DEBUG_INFO, "Set debug on")
		else:
			print(self._DEBUG_INFO, "Set debug off")


In [34]:
def test():
	'''Servo driver test on channel 1'''
	import time
	a = Servo(1)
	a.setup()
	for i in range(0, 180, 5):
		print(i)
		a.write(i)
		time.sleep(0.1)
	for i in range(180, 0, -5):
		print(i)
		a.write(i)
		time.sleep(0.1)
	for i in range(0, 91, 2):
		a.write(i)
		time.sleep(0.05)
	print(i)

In [39]:
test()

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
175
170
165
160
155
150
145
140
135
130
125
120
115
110
105
100
95
90
85
80
75
70
65
60
55
50
45
40
35
30
25
20
15
10
5
90


In [40]:
def install():
	all_servo = [0,0,0,0, 0,0,0,0, 0,0,0,0, 0,0,0,0]
	for i in range(16):
		all_servo[i] = Servo(i)
	for servo in all_servo:
		servo.setup()
		servo.write(90)

In [41]:
install()